In [48]:
import sqlite3
from datetime import datetime
import pandas as pd

In [49]:
DB_PATH = 'signal.db'

In [50]:
def init_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS signals(
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    timestamp TEXT, 
    ticker TEXT, 
    signal_type TEXT,
    current_price REAL, 
    entry_price REAL, 
    exit_price REAL,
    confidence REAL, 
    broker TEXT,
    comment TEXT
    )
    ''')
    conn.commit()
    conn.close()

In [63]:
def log_signal(ticker, signal_type, current_price, entry_price, exit_price, confidence, broker, comment):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    # Check for last signal
    c.execute('''
        SELECT signal_type FROM signals 
        WHERE ticker = ? 
        ORDER BY timestamp DESC 
        LIMIT 1
    ''', (ticker.upper(),))
    last = c.fetchone()

    if last and last[0] == signal_type.upper():
        print(f"Skipping duplicate signal for {ticker.upper()}: {signal_type.upper()}")
        conn.close()
        return
        
    c.execute('''
    INSERT INTO signals (timestamp,ticker, signal_type,current_price, entry_price, exit_price, confidence, broker,comment) 
    VALUES(?,?,?,?,?,?,?,?,?)
    ''',(datetime.now().isoformat(), ticker.upper(), signal_type.upper(),current_price, entry_price, exit_price, confidence, broker,comment))
    conn.commit()
    conn.close()

In [64]:
def get_signal_df():
    conn =sqlite3.connect(DB_PATH)
    df = pd.read_sql_query('SELECT * FROM signals ORDER BY timestamp DESC', conn)
    conn.close()
    return df


In [65]:
def export_signals_to_excel(path='signals_export.xlsx'):
    df = get_signals_df()
    df.to_excel(path, index=False)
    print(f"Exported to {path}")

In [71]:
# (timestamp,ticker, signal_type,current_price, entry_price, exit_price, confidence, broker,comment) 
init_db()
log_signal("BMO", "SELL", 114,115,102,'ABOVE 2 SD','IBKR','Should sell it')
log_signal("PGR", "BUY", 246,245,264,'BELOW 2 SD','IBKR','Should buy it')
log_signal("ABT", "BUY", 119,119,132,'BELOW 2 SD','IBKR','Should buy it')
log_signal("ELV", "BUY", 300,277,344,'BELOW 2 SD','IBKR','Should buy it')
log_signal("PG", "BUY", 154,152,157,'BELOW 2 SD','IBKR','Should buy it')

Skipping duplicate signal for BMO: SELL
Skipping duplicate signal for PGR: BUY
Skipping duplicate signal for ABT: BUY
Skipping duplicate signal for ELV: BUY


In [83]:
log_signal("DOMS", "BUY", 2364,2390,2590,'BELOW 1 SD','Zerodha','Bought but at 1 SD level should stick to code')
log_signal("UNITDSPR", "BUY", 1360,1382,1470,'BELOW 1 SD','Zerodha','Bought but at 1 SD level should stick to code')

log_signal("HCLTECH", "BUY", 1537,1500,1620,'BELOW 2 SD','Zerodha','wait for level')
log_signal("SAREGAMA", "BUY", 508,460,510,'BELOW 2 SD','Zerodha','wait for level')

log_signal("LUPIN", "BUY", 1923,1900,2190,'BELOW 2 SD','Zerodha','wait for level')
log_signal("FLUOROCHEM", "BUY", 3462,3240,3730,'BELOW 2 SD','Zerodha','wait for level')

log_signal("TRIVENI", "BUY", 366,360,400,'BELOW 1 SD','Zerodha','small trade; I want to test early detection of code ')


Skipping duplicate signal for DOMS: BUY
Skipping duplicate signal for UNITDSPR: BUY
Skipping duplicate signal for HCLTECH: BUY
Skipping duplicate signal for SAREGAMA: BUY
Skipping duplicate signal for LUPIN: BUY
Skipping duplicate signal for FLUOROCHEM: BUY


In [84]:
result= get_signal_df()

In [85]:
result

,id,timestamp,ticker,signal_type,current_price,entry_price,exit_price,confidence,broker,comment
0,13,2025-07-20T21:57:17.060085,TRIVENI,BUY,366.0,360.0,400.0,BELOW 1 SD,Zerodha,small trade; I want to test early detection of...
1,12,2025-07-20T21:52:22.670949,FLUOROCHEM,BUY,3462.0,3240.0,3730.0,BELOW 2 SD,Zerodha,wait for level
2,11,2025-07-20T21:52:22.663989,LUPIN,BUY,1923.0,1900.0,2190.0,BELOW 2 SD,Zerodha,wait for level
3,10,2025-07-20T21:52:22.659648,SAREGAMA,BUY,508.0,460.0,510.0,BELOW 2 SD,Zerodha,wait for level
4,9,2025-07-20T21:52:22.656975,HCLTECH,BUY,1537.0,1500.0,1620.0,BELOW 2 SD,Zerodha,wait for level
5,8,2025-07-20T21:52:22.654541,UNITDSPR,BUY,1360.0,1382.0,1470.0,BELOW 1 SD,Zerodha,Bought but at 1 SD level should stick to code
6,7,2025-07-20T21:52:22.645779,DOMS,BUY,2364.0,2390.0,2590.0,BELOW 1 SD,Zerodha,Bought but at 1 SD level should stick to code
7,6,2025-07-20T19:43:59.295747,COST,BUY,950.0,900.0,100.0,BELOW 2 SD,IBKR,Should buy it
8,5,2025-07-20T19:41:32.207164,PG,BUY,154.0,152.0,157.0,BELOW 2 SD,IBKR,Should buy it
9,4,2025-07-20T19:40:17.033272,ELV,BUY,300.0,277.0,344.0,BELOW 2 SD,IBKR,Should buy it
